In [1]:
import os
import time
import numpy as np
import pandas as pd
import yfinance as yf
import pickle
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
import argparse
import json


In [2]:
np.random.seed(42)
tf.random.set_seed(42)

CACHE_DIR = "cache"
MODEL_DIR = "saved_models"
os.makedirs(CACHE_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
EPSILON = 0.005
MIN_CANDIDATES = 2
CACHE_DURATION = 86400  # 1 day in seconds
LSTM_EPOCHS = 5         # Reduced training epochs
LOOK_BACK = 50          # Fixed look-back window

In [3]:
def get_historical_data_cached(ticker, period="1y", interval="1d", cache_duration=CACHE_DURATION):
    """
    Fetch historical data from yfinance, caching the result in CACHE_DIR for 'cache_duration' seconds.
    """
    cache_filename = os.path.join(CACHE_DIR, f"{ticker}_{period}_{interval}.pkl")
    now = time.time()
    if os.path.exists(cache_filename):
        if now - os.path.getmtime(cache_filename) < cache_duration:
            try:
                return pd.read_pickle(cache_filename)
            except Exception as e:
                print(f"Error reading cache for {ticker}: {e}")
    df = yf.download(ticker, period=period, interval=interval)
    df.dropna(inplace=True)
    df.to_pickle(cache_filename)
    return df

# Use cached function for data fetching
get_historical_data = get_historical_data_cached

In [4]:
def load_trained_model(ticker, cache_duration=CACHE_DURATION):
    """
    Loads a previously trained model and scaler if they exist and are fresh.
    """
    model_path = os.path.join(MODEL_DIR, f"{ticker}_lstm_model.keras")
    scaler_path = os.path.join(MODEL_DIR, f"{ticker}_scaler.pkl")
    if os.path.exists(model_path) and os.path.exists(scaler_path):
        if time.time() - os.path.getmtime(model_path) < cache_duration:
            try:
                model = load_model(model_path)
                with open(scaler_path, "rb") as f:
                    scaler = pickle.load(f)
                return model, scaler, LOOK_BACK
            except Exception as e:
                print(f"Error loading model for {ticker}: {e}")
    return None, None, None

In [5]:
def save_trained_model(ticker, model, scaler):
    """
    Saves model and scaler to MODEL_DIR.
    """
    model_path = os.path.join(MODEL_DIR, f"{ticker}_lstm_model.keras")
    scaler_path = os.path.join(MODEL_DIR, f"{ticker}_scaler.pkl")
    model.save(model_path)
    with open(scaler_path, "wb") as f:
        pickle.dump(scaler, f)

In [6]:
def train_lstm_model(ticker, epochs=LSTM_EPOCHS, batch_size=32):
    """
    Loads a pre-trained model if fresh; otherwise trains a new LSTM model with reduced epochs.
    """
    model, scaler, look_back = load_trained_model(ticker)
    if model is not None:
        return model, scaler, look_back

    df = get_historical_data(ticker, period="1y", interval="1d")
    if df is None or df.empty:
        return None, None, None

    data = df['Close'].values.reshape(-1, 1)
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data)
    if len(scaled_data) <= LOOK_BACK:
        print(f"Not enough data to train {ticker} (need > {LOOK_BACK} points).")
        return None, None, None

    X, y = [], []
    for i in range(LOOK_BACK, len(scaled_data)):
        X.append(scaled_data[i - LOOK_BACK:i, 0])
        y.append(scaled_data[i, 0])
    X, y = np.array(X), np.array(y)
    X = X.reshape(X.shape[0], X.shape[1], 1)

    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(X.shape[1], 1)),
        Dropout(0.2),
        LSTM(50),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    callbacks = [
        EarlyStopping(monitor='loss', patience=2, restore_best_weights=True),
        ModelCheckpoint(os.path.join(MODEL_DIR, f"{ticker}_lstm_best.keras"), 
                        monitor='loss', save_best_only=True)
    ]
    model.fit(X, y, epochs=epochs, batch_size=batch_size, verbose=1, callbacks=callbacks)
    save_trained_model(ticker, model, scaler)

In [7]:
def forecast_lstm_weekly(ticker, model, scaler, look_back=LOOK_BACK, forecast_weeks=5, days_per_week=5):
    """
    Forecasts prices on a weekly basis by simulating 'days_per_week' daily forecasts and taking the final predicted price for each week.
    """
    df = get_historical_data(ticker, period="1y", interval="1d")
    if df is None or df.empty:
        return None
    data = df['Close'].values.reshape(-1, 1)
    scaled_data = scaler.transform(data)
    last_seq = scaled_data[-look_back:]
    weekly_forecasts = []
    for _ in range(forecast_weeks):
        # Simulate a week of trading days
        for _ in range(days_per_week):
            X_input = last_seq.reshape(1, look_back, 1)
            pred = model.predict(X_input)[0][0]
            last_seq = np.append(last_seq[1:], [[pred]], axis=0)
        weekly_forecasts.append(pred)
    forecasted_prices = scaler.inverse_transform(np.array(weekly_forecasts).reshape(-1, 1)).flatten()
    return forecasted_prices


In [8]:
def compute_lstm_return(ticker, forecast_weeks=52, days_per_week=5):
    """
    Computes short-term return using weekly forecasts.
    """
    model, scaler, look_back = train_lstm_model(ticker, epochs=5, batch_size=32)
    if model is None:
        return None
    pred_prices = forecast_lstm_weekly(ticker, model, scaler, look_back, forecast_weeks, days_per_week)
    if pred_prices is None or len(pred_prices) == 0:
        return None
    final_pred_price = float(pred_prices[-1])
    df_today = get_historical_data(ticker, period="1d", interval="1d")
    if df_today is None or df_today.empty:
        return None
    current_price = float(df_today['Close'].iloc[-1])
    if current_price <= 0:
        return None
    short_term_return = (final_pred_price - current_price) / current_price
    return short_term_return

In [9]:
def get_extended_universe():
    # Define an extended list with at least 50 stocks.
    universe = [
        "AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "NVDA", "JPM", "V", "DIS", "KO",
        "INTC", "CSCO", "ORCL", "IBM", "BA", "GE", "WMT", "PG", "NKE", "CRM",
        "ADBE", "MMM", "HON", "AXP", "UPS", "PM", "CAT", "MCD", "WFC", "C",
        "GS", "BK", "SBUX", "CVX", "XOM", "PFE", "MRK", "LLY", "ABT", "T",
        "VZ", "CMCSA", "NFLX", "TWTR", "SNAP", "UBER", "LYFT", "F", "GM", "AMD"
    ]
    return universe

In [10]:
def recommend_portfolio(risk_level, income, goal_duration, monthly_investment, target_amount, sector_cap=0.30):
    universe = get_extended_universe()
    filtered_scores = {}
    for ticker in universe:
        short_ret = compute_lstm_return(ticker, forecast_weeks=1, days_per_week=5)
        if short_ret is None:
            continue
        if short_ret > 0:
            filtered_scores[ticker] = (short_ret, short_ret)  # Using the same short_ret for both score and expected return

    if not filtered_scores or len(filtered_scores) < MIN_CANDIDATES:
        print("Not enough diversified candidates found.")
        return (None, None)

    stock_info = {}
    for ticker in filtered_scores.keys():
        try:
            info = yf.Ticker(ticker).info
            market_cap = info.get('marketCap', 1)
            sector = info.get('sector', 'Unknown')
            stock_info[ticker] = {'market_cap': market_cap, 'sector': sector}
        except Exception as e:
            print(f"Error fetching info for {ticker}: {e}")
            stock_info[ticker] = {'market_cap': 1, 'sector': 'Unknown'}

    weighted_scores = {ticker: filtered_scores[ticker][0] * stock_info[ticker]['market_cap']
                       for ticker in filtered_scores.keys()}
    total_weight = sum(weighted_scores.values())
    if total_weight == 0:
        return (None, None)
    initial_alloc = {ticker: weighted_scores[ticker] / total_weight for ticker in weighted_scores}

    sector_alloc = {}
    for ticker, weight in initial_alloc.items():
        sector = stock_info[ticker]['sector']
        sector_alloc[sector] = sector_alloc.get(sector, 0) + weight

    capped_alloc = initial_alloc.copy()
    excess = 0.0
    for ticker, weight in initial_alloc.items():
        sector = stock_info[ticker]['sector']
        if sector_alloc[sector] > sector_cap:
            capped_weight = weight * (sector_cap / sector_alloc[sector])
            excess += (weight - capped_weight)
            capped_alloc[ticker] = capped_weight

    new_sector_alloc = {}
    for ticker, weight in capped_alloc.items():
        sector = stock_info[ticker]['sector']
        new_sector_alloc[sector] = new_sector_alloc.get(sector, 0) + weight
    
    eligible = {ticker: weight for ticker, weight in capped_alloc.items()
                if new_sector_alloc.get(stock_info[ticker]['sector'], 0) < sector_cap}
    if eligible:
        total_eligible = sum(eligible.values())
        for ticker in eligible:
            add_weight = excess * (eligible[ticker] / total_eligible)
            capped_alloc[ticker] += add_weight

    final_total = sum(capped_alloc.values())
    final_alloc = {ticker: weight / final_total for ticker, weight in capped_alloc.items() if weight > 0}
    final_alloc = {ticker: weight for ticker, weight in final_alloc.items() if weight >= EPSILON}
    total_final = sum(final_alloc.values())
    if total_final == 0:
        return (None, None)
    final_alloc = {ticker: weight / total_final for ticker, weight in final_alloc.items()}

    portfolio_expected_return = sum(final_alloc[ticker] * filtered_scores[ticker][1] for ticker in final_alloc)
    if portfolio_expected_return <= 0:
        print("Overall portfolio expected return is non-positive.")
        return (None, None)

    print("Selected Stocks & Sectors:")
    for ticker in final_alloc:
        print(f"{ticker}: Sector: {stock_info[ticker]['sector']}, Weight: {final_alloc[ticker]*100:.1f}%")
    print(f"\nEstimated Portfolio Annual Return: {portfolio_expected_return*100:.2f}%")

    r_monthly = (1 + portfolio_expected_return)**(1/12) - 1
    n = goal_duration * 12
    required_PMT = target_amount * r_monthly / ((1 + r_monthly)**n - 1)
    print(f"Target Goal: ${target_amount:,.2f} in {goal_duration} years")
    print(f"Required Monthly Investment (estimated): ${required_PMT:,.2f}")
    print(f"Your current Monthly Investment: ${monthly_investment:,.2f}")
    if monthly_investment < required_PMT:
        print(f"You may need an additional ${(required_PMT - monthly_investment):,.2f} monthly.")
    else:
        print("Your current monthly investment should suffice to reach your goal.")
    print(f"\nUser Inputs -> Risk Level: {risk_level}, Income: ${income}, "
          f"Goal Duration: {goal_duration} years, Monthly Investment: ${monthly_investment}")
    
    return final_alloc, required_PMT

In [11]:
def main():
    print("=== AI-Powered Investment Insights ===")
    risk_level = input("Enter your risk level (Conservative, Moderate, Aggressive): ").strip()
    income = float(input("Enter your monthly income ($): "))
    goal_duration = int(input("Enter your goal duration (years): "))
    monthly_investment = float(input("Enter your current monthly investment amount ($): "))
    target_amount = float(input("Enter your target goal amount ($): "))
    
    allocation, required_PMT = recommend_portfolio(risk_level, income, goal_duration, monthly_investment, target_amount)
    if allocation is not None:
        print("\n--- Recommended Diversified Portfolio ---")
        for ticker, weight in allocation.items():
            print(f"{ticker}: {weight*100:.1f}%")
    else:
        print("Unable to generate a portfolio recommendation with the current data.")

if __name__ == '__main__':
    main()

=== AI-Powered Investment Insights ===


Enter your risk level (Conservative, Moderate, Aggressive):  Moderate
Enter your monthly income ($):  5000
Enter your goal duration (years):  5
Enter your current monthly investment amount ($):  500
Enter your target goal amount ($):  150000


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


/var/folders/tc/fjfxhp_j0yv_1k8tf69zd0m80000gn/T/ipykernel_67284/1021540490.py:15: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  current_price = float(df_today['Close'].iloc[-1])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


...

/var/folders/tc/fjfxhp_j0yv_1k8tf69zd0m80000gn/T/ipykernel_67284/1021540490.py:15: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  current_price = float(df_today['Close'].iloc[-1])
[*********************100%***********************]  1 of 1 completed

Epoch 1/5



/Users/dhanyavenkatesh/Investomate/myenv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 0.1840
Epoch 2/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0394
Epoch 3/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0251
Epoch 4/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0191
Epoch 5/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0204


TypeError: cannot unpack non-iterable NoneType object